## Margin Recommender by Clustering

In this notebook we will try to predict recommended margin range for when a customer wants to by a new product by using unsupervised learning. 
More specifically we will first cluster products and then customers. After the clustering is performed we can calcuate the upper and lower bound for recommended margin with the following formula: 



We start off by importing the necessary libraries, and reading in the data.

In [248]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
from itertools import combinations
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer



### Helper Functions
This section contains a variety of important functions used throughout the project. 

In [249]:
def get_coefficient_variation(data, feature): 
    '''
    The function calculates the coefficient of variation (CV) of the "Margin" for each category in the specified feature.

    Returns a sorted DataFrame with the mean, standard deviation, and CV for each category.
    '''
    
    grouped_data = data.groupby(feature)
    mean, std = grouped_data["Margin"].mean(), grouped_data["Margin"].std()
    df = pd.concat([mean, std], axis = 1).reset_index()
    df.columns = [feature, "Mean", "Std"]
    df["CoefficientOfVariation"] = (df["Std"]/df["Mean"])**2

    return df.sort_values(by = "CoefficientOfVariation")


In [250]:


def split_data(feature, data):
    '''
    Splits a DataFrame into multiple DataFrames based on the unique values of a specified categorical feature.

    Parameters:
    - feature: The column name to split the DataFrame by.
    - data: The DataFrame to split.

    Returns:
    - A dictionary where keys are unique feature values and values are the corresponding DataFrames.
    '''

    categories = list(data[feature].unique())
    dataframes = {}

    for c in categories: 
        df = data[data[feature] == c]
        dataframes[c] = df
        
    return dataframes


In [251]:
from datetime import datetime

'''
This cell contains all functions that are used to create new features used by the KMeans
algorithms to group products in clusters. 
More functions are needed to create more features to achieve better clustering results. 
'''

def add_average_margin_feature(dataframe):
    means = dataframe.groupby("ProductName")["Margin"].mean().reset_index()
    means.columns = ["ProductName", "MeanMargin"]
    new_df = pd.merge(means, dataframe, on = "ProductName")
    return new_df 
    

def add_average_cost_per_unit_feature(dataframe):
    dataframe.loc[:, "CostPerUnit"] = dataframe["Cost"]/dataframe["Quantity"]
    means = dataframe.groupby("ProductName")["CostPerUnit"].mean().reset_index()
    means.columns = ["ProductName", "MeanCostPerUnit"]
    new_df = pd.merge(means, dataframe, on = "ProductName")
    return new_df 

def add_average_sales_in_past_per_product(n_months, dataframe):
    dataframe["OrderDate"] = pd.to_datetime(dataframe["OrderDate"])
    last_date = dataframe["OrderDate"].max()
    n_months_ago = last_date - pd.DateOffset(months = n_months)
    filtered_data = dataframe[dataframe["OrderDate"] >= n_months_ago]
    meanSales = filtered_data.groupby("ProductName")["Sales"].mean().reset_index()
    meanSales.columns = ["ProductName", "AverageSalesPastMonths"]
    new_df = pd.merge(meanSales, dataframe, on = "ProductName")
    return new_df

def number_of_orders_per_product(dataframe): 
    product_order_counts = dataframe.groupby("ProductID")["OrderID"].count().reset_index()
    product_order_counts = product_order_counts.rename(columns = {"OrderID" : "NumberOfOrders"})
    dataframe = dataframe.merge(product_order_counts, on = "ProductID", how = "left")
    return dataframe
    
def add_total_sales_for_product(dataframe): 
    product_sales = dataframe.groupby("ProductID")["Sales"].sum().reset_index()
    product_sales.columns = ["ProductID", "TotalSales"]
    new_df = pd.merge(dataframe, product_sales, on = "ProductID", how = "left")
    return new_df

def add_total_cost_for_product(dataframe): 
    product_cost = dataframe.groupby("ProductID")["Cost"].sum().reset_index()
    product_cost.columns = ["ProductID", "TotalCost"]
    new_df = pd.merge(dataframe, product_cost, on = "ProductID", how = "left")
    return new_df

    

# Group products based on sales. Use the TotalRevenue. 
# Start by picking two features and see if there is any clustering. For example revenue and avg margin, or margin and number of orders. 
# Do not use average but sum. Also try changing the scaling method. 

In [252]:
filepath = 'CogitoXInfor-h24/ECOMMRecords2020.csv'
ecommerce_df = pd.read_csv(filepath)

ecommerce_df["Cost"] = (ecommerce_df["Sales"] - ecommerce_df["Profit"])/ecommerce_df["Quantity"] 
ecommerce_df["Margin"] = ecommerce_df["Profit"]/ecommerce_df["Sales"]


In [253]:
def feature_engineering_preprocessing(dataframes: dict, scaler) -> dict:
    '''
    The function applies multiple feature engineering steps to each dataframe in the input dictionary. 

    It then returns a dictionary with the engineered dataframes. 
    '''
    result = {}
    for category in dataframes.keys():
        df = dataframes[category]

        df = number_of_orders_per_product(df)
        df = add_total_sales_for_product(df)
        df = add_total_cost_for_product(df)
        df = add_average_margin_feature(df)

        result[category] = df

    return result



In [254]:
def find_optimal_k(dataframes : dict, engineered_features : list): 
    '''
    Identifies the optimal number of clusters for KMeans using the elbow method and plotting the silhouette score. 

    Returns a plot of the interia score and silhouette score versus number of clusters for each of the dataframes 
    in the input dictionary. 
    '''
    scaler = MinMaxScaler()

    ks = [x for x in range(2, 11)]
    inertia_scores = []
    silhouette_scores = []
    
    for category in dataframes.keys(): 
        data = scaler.fit_transform(dataframes[category][engineered_features])
        inertia = []
        silhouette = []
        for k in ks: 
            model = KMeans(n_clusters = k, random_state = 42, init = 'k-means++')
            predicted = model.fit_predict(data)
            inertia.append(model.inertia_)
            silhouette.append(silhouette_score(data, predicted))

        inertia_scores.append(inertia)
        silhouette_scores.append(silhouette)
    
    fig, ax = plt.subplots(nrows = len(dataframes.keys()), ncols = 2, figsize = (15, 14), squeeze=False)
    for i in range(len(dataframes.keys())): 
        ax[i, 0].plot(ks, inertia_scores[i])
        ax[i, 1].plot(ks, silhouette_scores[i])
        ax[i, 0].set_xlabel("Number of Clusters")
        ax[i, 0].set_ylabel("Inertia")
        ax[i, 1].set_xlabel("Number of Clusters")
        ax[i, 1].set_ylabel("Silhouette Score")
        fig.text(0.5, 0.95 - (i*0.3), f"{list(dataframes.keys())[i]}", ha = 'center', fontsize = 14, weight = "bold")

    plt.subplots_adjust(hspace=0.5)
    plt.show()

In [255]:
def visualize_clusters(model, data, title):
    '''
    The function visualizes clusters produced by a clustering model in 2D space, by 
    using t-SNE for dimensionality reduction. 

    Returns a 2D scatter plot of the data points colored by their predicted cluster labels. 
    '''
    embedding = TSNE(n_components=2,
        init="pca",
        max_iter=500,
        n_iter_without_progress=150,
        perplexity= 20,
        random_state=0)   

    data_2D = embedding.fit_transform(data)
    
    labels = model.labels_
    cmap = plt.get_cmap('tab10', model.n_clusters)

    unique_labels = set(labels)
    fig, ax = plt.subplots(figsize = (15, 10))

    for l in unique_labels: 
        cluster = data_2D[labels == l]
        ax.scatter(cluster[:, 0], cluster[:, 1], color = cmap(l), label = f"Cluster {l}")

    ax.legend()
    ax.set_title(title)
    plt.show()



In [256]:
def cluster_product_data(dataframes_scaled : dict, num_clusters : dict, cluster_features : list ,vizualize = False): 
    '''
    For each dataframe in the dictionary that contains the scaled data, and the features that we proceed with we fit a KMeans
    model to each of the dataframes, with the number of clusters specified in the num_clusters dictionary. 
    '''

    scaler = MinMaxScaler()

    for category in dataframes_scaled.keys(): 
        cluster_data = scaler.fit_transform(dataframes_scaled[category][cluster_features])
        model = KMeans(n_clusters = num_clusters[category], init = "k-means++", random_state=42)
        predicted = model.fit_predict(cluster_data)
        
        if vizualize: visualize_clusters(model, cluster_data, category) 

        dataframes_scaled[category]["ProductCluster"] = predicted


In [257]:
def calculate_total_revenue(customer_clustering_data : dict):
    result = {}

    for category in customer_clustering_data.keys():
        data = customer_clustering_data[category]
        cluster_revenues_customer = data.groupby(["CustomerID", "ProductCluster"])["TotalSales"].sum()
        cluster_revenues_customer = cluster_revenues_customer.unstack(fill_value = 0)
        cluster_revenues_customer.columns = [f"totalRevenue_PC_{col}" for col in cluster_revenues_customer]
        cluster_revenues_customer = cluster_revenues_customer.reset_index()

        result[category] = pd.merge(data, cluster_revenues_customer, on = "CustomerID", how = "left")
    
    return result


In [258]:
def cluster_customer_data(customer_clustering_data : dict, n_clusters : dict, cluster_features, vizualise = False):

    scaler = MinMaxScaler()

    for category in customer_clustering_data.keys():
        cluster_data = scaler.fit_transform(customer_clustering_data[category][cluster_features])
        model = KMeans(n_clusters=n_clusters[category], init = "k-means++", random_state=42)
        predictions = model.fit_predict(cluster_data)

        if vizualise: visualize_clusters(model, cluster_data, category)

        customer_clustering_data[category]["CustomerCluster"] = predictions
        


In [259]:
import numpy as np

# Sample data: array of tuples with 'cluster_id' and 'margin'
# bytt ut data med customercluster info

def get_lower_upper_margin(data, customer_cluster): 
    # Extract 'margin' and 'cluster_id' arrays
    margins = data['margin']
    cluster_ids = data['cluster_id']

    def percentile_rank(value, data_array):
        """Calculate the percentile rank of a value within an array."""
        sorted_array = np.sort(data_array)
        count = np.searchsorted(sorted_array, value, side='left')
        percentile = (count / len(data_array)) * 100
        return percentile

    # Get unique cluster IDs
    unique_clusters = np.unique(cluster_ids)

    # Dictionary to store results
    cluster_percentiles = {}

    # Calculate 40th and 75th percentiles for each cluster
    for cluster in unique_clusters:
        # Filter margins for the current cluster
        cluster_margins = margins[cluster_ids == cluster]

        # Calculate 40th and 75th percentiles within the cluster
        percentile_40 = np.percentile(cluster_margins, 40)
        percentile_75 = np.percentile(cluster_margins, 75)

        # Calculate percentile ranks for the overall data
        percentile_40_rank = percentile_rank(percentile_40, margins)
        percentile_75_rank = percentile_rank(percentile_75, margins)

        # Store the results
        cluster_percentiles[cluster] = {
            '40th_percentile_margin': percentile_40,
            '40th_percentile_rank': percentile_40_rank,
            '75th_percentile_margin': percentile_75,
            '75th_percentile_rank': percentile_75_rank
        }

    lower_margin = cluster_percentiles[customer_cluster]['40th_percentile_margin']
    upper_margin = cluster_percentiles[customer_cluster]['75th_percentile_margin']

    return lower_margin, upper_margin





In [260]:
def calculate_price(margin, cost):
    return cost/(1-margin)

In [261]:

#category = get_coefficient_variation(data, "Category") # We found that the it is best to split the data on category by looking at the coefficient of variation. 
#sub_category = get_coefficient_variation(data, "SubCategory")

### Price Recommender Pipeline

In [262]:
def product_clustering_pipeline(ecommerce_df : pd.DataFrame) -> dict:
    '''
    The function takes the original dataframe for US ECOMM RECORDS 2020 and first splits the dataframe
    based on Category. Then appropriate feature engineering to get a dataframe ready for clustering using KMeans.

    The KMeans algorithm is applied and (you select the number of appropriate clusters based on inertia and silhouette graphs)
    and the identified cluster for each row is added as a feature. 

    This accquired dataframe is returned by this function and can be used as input for customer clustering. 
    '''
    scaler = MinMaxScaler()
    ecommerce_grouped = split_data(feature = "Category", data = ecommerce_df)
    ecommerce_engineered = feature_engineering_preprocessing(ecommerce_grouped, scaler)
    engineered_features = ["TotalCost",	"TotalSales", "NumberOfOrders", "MeanMargin"]
    num_clusters = {"Technology" : 5, "Furniture" : 5, "Office Supplies" : 5}
    #find_optimal_k(ecommerce_engineered, engineered_features)
    #num_clusters = input("Enter optimal number of clusters for each category according to the plot.")
    #num_clusters = {key.strip(): int(value) for key, value in (item.split(":") for item in num_clusters.split(", "))}

    cluster_product_data(ecommerce_engineered, num_clusters, engineered_features)
    
    return ecommerce_engineered



In [263]:
def customer_clustering_pipeline(customer_clustering_data : dict) -> dict:
    customer_clustering_data = calculate_total_revenue(customer_clustering_data)
    clustering_features = ["totalRevenue_PC_0",	"totalRevenue_PC_1",	"totalRevenue_PC_2",	"totalRevenue_PC_3",	"totalRevenue_PC_4"]
    num_clusters = {"Technology" : 5, "Furniture" : 5, "Office Supplies" : 5}
    cluster_customer_data(customer_clustering_data, num_clusters, clustering_features)

    return customer_clustering_data

In [264]:
def compute_recommended_price(complete_data : dict, customer_id : str, product_id : str):
    unsplit_complete_data = pd.concat(complete_data.values(), ignore_index=True)
    category = unsplit_complete_data[unsplit_complete_data["ProductID"] == product_id]["Category"].unique()[0]
    product_cluster = unsplit_complete_data[unsplit_complete_data["ProductID"] == product_id]["ProductCluster"].unique()[0]
    print(product_cluster)

    percentile_data = {}
    for category in complete_data.keys():
        data = complete_data[category]
        data = data[data["ProductCluster"] == product_cluster]
        print(data)
        data = data[["CustomerCluster", "Margin"]]
        data_array = np.array(list(data.itertuples(index = False, name = None)), 
                                 dtype=[("cluster_id", int), ("margin", float)])
        percentile_data[category] = data_array
    
    
    data = complete_data[category]
    cost = unsplit_complete_data[unsplit_complete_data["ProductID"] == product_id]["Cost"].unique()[0]
    
    customer_cluster = data.loc[data["CustomerID"] == customer_id, "CustomerCluster"].values[0]

    margin_lower, margin_upper = get_lower_upper_margin(percentile_data[category], customer_cluster)
    lower_price, upper_price = calculate_price(margin_lower, cost), calculate_price(margin_upper, cost)
    recommended_price = calculate_price((margin_lower + margin_upper)/2, cost)

    return lower_price, upper_price, recommended_price




In [265]:
def pipeline(customer_id, product_id):
    filepath = 'CogitoXInfor-h24/ECOMMRecords2020.csv'
    ecommerce_df = pd.read_csv(filepath)

    ecommerce_df["Cost"] = (ecommerce_df["Sales"] - ecommerce_df["Profit"])/ecommerce_df["Quantity"] 
    ecommerce_df["Margin"] = ecommerce_df["Profit"]/ecommerce_df["Sales"]

    customer_clustering_data =  product_clustering_pipeline(ecommerce_df)
    complete_data = customer_clustering_pipeline(customer_clustering_data)
    print(complete_data["Technology"].columns)
    lower_price, upper_price, recommended_price = compute_recommended_price(complete_data, customer_id, product_id)


    print(f"For product {product_id} and the customer {customer_id} we recommend the following:")
    print(f"Lower sales price: {lower_price:.2f}")
    print(f"Upper sales price: {upper_price:.2f}")
    print(f"Recommended sales price: {recommended_price:.2f}")

In [266]:
pipeline("JF-15295", "FUR-FU-10001215")

Index(['ProductName', 'MeanMargin', 'OrderDate', 'RowID', 'OrderID',
       'ShipMode', 'CustomerID', 'Segment', 'Country', 'City', 'State',
       'PostalCode', 'Region', 'ProductID', 'Category', 'SubCategory', 'Sales',
       'Quantity', 'Discount', 'Profit', 'Cost', 'Margin', 'NumberOfOrders',
       'TotalSales', 'TotalCost', 'ProductCluster', 'totalRevenue_PC_0',
       'totalRevenue_PC_1', 'totalRevenue_PC_2', 'totalRevenue_PC_3',
       'totalRevenue_PC_4', 'CustomerCluster'],
      dtype='object')
0
                                           ProductName  MeanMargin  \
3                             24-Hour Round Wall Clock      0.4300   
4                             24-Hour Round Wall Clock      0.4300   
11   3M Polarizing Task Lamp with Clamp Arm Light Gray      0.2600   
12   3M Polarizing Task Lamp with Clamp Arm Light Gray      0.2600   
13                          6 Cubicle Wall Clock Black     -0.1200   
..                                                 ...         ... 